In [ ]:
import argparse
import numpy as np

# Main C3 objects
from c3.libraries import constants
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.model import Model as Mdl

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.libraries.chip as chip
import c3.signal.pulse as pulse
import c3.libraries.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.fidelities as fidelities
import c3.utils.qt_utils as qt_utils
from c3.optimizers.optimalcontrol import OptimalControl

#%matplotlib widget

from four_level_transmons.utilities import *
from four_level_transmons.plotting import *
from four_level_transmons.custom_envelopes import *
from four_level_transmons.DataOutput import DataOutput

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()
output_dir = args.output
#output_dir = "../output"

In [ ]:
def printSignal(exper: Experiment, qubit: chip.Qubit,
                gate: gates.Instruction, pwcTimes: np.array,
                output: DataOutput):
    # generate signal
    signal = generateSignal(exper, gate, qubit)
    ts = signal["ts"].numpy()
    values = signal["values"].numpy()

    # plot and save
    peakFrequencies, peakValues = findFrequencyPeaks(ts, values, 4)
    print("peaks: ", np.sort(peakFrequencies))
    output.save([ts, values], "signal")
    envelope = getEnvelope(exper.pmap.generator, gate, "d1")
    plotSignalAndSpectrum(ts, values, envelope=envelope, pwcTimes=pwcTimes,
                          filename=output.createFileName("signal", "png"))


def printTimeEvolution(exper: Experiment, init: tf.Tensor, gate: gates.Instruction,
                       labels: List[str], output: DataOutput):
    populations = calculatePopulation(exper, init, [gate.get_key()])
    output.save(populations, "population")
    plotPopulation(exper, populations, sequence=[gate.get_key()],
                   labels=labels, filename=output.createFileName("population"))


def printPropagator(exper: Experiment, gate: gates.Instruction,
                    labels: List[str], output: DataOutput):
    U = exper.propagators[gate.get_key()]
    output.save(U, "propagator")

    plotComplexMatrix(U, xlabels=labels, ylabels=labels, filename=output.createFileName("propagator"))
    plotComplexMatrixAbsOrPhase(U, xlabels=labels, ylabels=labels, phase=True, filename=output.createFileName("propagator_phase"))
    plotComplexMatrixAbsOrPhase(U, xlabels=labels, ylabels=labels, phase=False, filename=output.createFileName("propagator_abs"))

In [ ]:
# Initialise the qubits and drive lines
qubit_levels = 5
qubit_frequency = 5e9
anharmonicity = -300e6
t1 = 25e-6
t2star = 35e-6
qubit_temp = 50e-3
level_labels = ["$|0,0\\rangle$", "$|0,1\\rangle$", "$|1,0\\rangle$", "$|1,1\\rangle$", "leakage"]
output = DataOutput(output_dir, file_suffix='before')

qubit = createQubits([qubit_levels], [qubit_frequency], [anharmonicity], [t1],
                      [t2star], qubit_temp)[0]
drive = createDrives([qubit])[0]

In [ ]:
# Create the model
model = Mdl([qubit], [drive])
model.set_lindbladian(False)
model.set_dressed(True)

energies = qubit.get_Hamiltonian().numpy().diagonal().real
print("energies: ", energies)
print("transition frequencies: ", [energies[i+1] - energies[i] for i in range(len(energies)-1)])

In [ ]:
# Create the generator
sim_res = 100e9
awg_res = 2e9
v2hz = 1e9
generator = createGenerator([drive], useDrag=True)

In [ ]:
# Envelopes and carriers
t_final = 30e-9
sideband = 50e6
carrier_freqs = [4.7e9]
numPWCPieces = 30

gaussian_envelope = createPWCDoubleGaussianPulse(
    t_final=t_final,
    sigma=t_final / 15,
    sigma2=t_final / 30,
    relative_amp=2,
    num_pieces=numPWCPieces
)
nodrive_envelope = pulse.Envelope(
    name="no_drive",
    params={
        "t_final": Qty(value=t_final, min_val=0.5 * t_final, max_val=1.5 * t_final, unit="s")
    },
    shape=envelopes.no_drive
)

qubit_freqs = model.get_qubit_freqs()
carriers = createCarriers(carrier_freqs, sideband)

In [ ]:
# Gate instructions
ideal_gate = qt_utils.np_kron_n([constants.GATES["rz90p"], constants.Id])
plotComplexMatrix(ideal_gate, xlabels=level_labels[:4], ylabels=level_labels[:4],
                  filename=output.createFileName("ideal_gate", "png"))
gate = createSingleQubitGate("rz90p_q1", [drive], carriers, 0, t_final,
                                          gaussian_envelope, nodrive_envelope, sideband,
                                          ideal=ideal_gate)

In [ ]:
# Set up the experiment
parameter_map = PMap(instructions=[gate], model=model, generator=generator)
exp = Exp(pmap=parameter_map)
exp.set_opt_gates([gate.get_key()])

printSignal(exp, qubit, gate, pwcTimes=tf.linspace(0.0, t_final, numPWCPieces), output=output)

unitaries = exp.compute_propagators()
printPropagator(exp, gate, level_labels, output)

In [ ]:
# Specify the initial state
psi_init = [[0] * model.tot_dim]
psi_init[0][0] = 1
init_state = tf.transpose(tf.constant(psi_init, tf.complex128))
sequence = [gate.get_key()]

exp.compute_propagators()
printTimeEvolution(exp, init_state, gate, level_labels, output)
#plotSplittedPopulation(exp, init_state, sequence)

In [ ]:
# Specify the parameters to be optimised and initialise the optimiser
parameter_map.set_opt_map([
    [(gate.get_key(), "d1", gaussian_envelope.name, "amp")],
    [(gate.get_key(), "d1", gaussian_envelope.name, "freq_offset")],
    [(gate.get_key(), "d1", gaussian_envelope.name, "xy_angle")],
    [(gate.get_key(), "d1", gaussian_envelope.name, "delta")],
    [(gate.get_key(), "d1", gaussian_envelope.name, "t_final")],
    [(gate.get_key(), "d1", gaussian_envelope.name, "inphase")],
    [(gate.get_key(), "d1", carriers[0].name, "freq")],
    [(gate.get_key(), "d1", carriers[0].name, "framechange")],
])
parameter_map.print_parameters()

opt = OptimalControl(
    dir_path=output.getDirectory(),
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=[qubit.name],
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={"maxfun": 600},
    run_name="rx90_q1",
    fid_func_kwargs={
        "active_levels": 4
    }
)
exp.set_opt_gates([gate.get_key()])
opt.set_exp(exp)
opt.set_callback(lambda index, fidelity: print(index, fidelity))

In [ ]:
# Run the optimisation
opt.optimize_controls()
opt.current_best_goal

In [ ]:
# Print and plot results
parameter_map.print_parameters()

output = DataOutput(output_dir, file_suffix='after')
printSignal(exp, qubit, gate, pwcTimes=tf.linspace(0.0, t_final, numPWCPieces), output=output)
printPropagator(exp, gate, level_labels, output)
printTimeEvolution(exp, init_state, gate, level_labels, output)